# LIMPIEZA

In [ ]:
import pandas as pd
import numpy as np
import re


sheets = ["CENTRO","SUROESTE","NORTE","SUROESTE2","SURESTE3","NORESTE2"]


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


## 2022

In [2]:
# ============================
# 1. CARGAR DATOS
# ============================

raw_data = pd.read_excel(
    "DATOS_HISTORICOS_2022.xlsx",
    sheet_name=sheets
)

# Remover columnas que no usarás
for name in sheets:
    raw_data[name] = raw_data[name].drop(columns=["RAINF","RH","PRS"])


# ============================
# 2. RANGOS PERMITIDOS
# ============================

ranges = {
    "CO": (0, 8),
    "NO": (0, 400),
    "NO2": (0, 175),
    "NOX": (0, 420),
    "O3": (0, 160),
    "PM10": (0, 999),
    "PM2.5": (0, 450),
    "SO2": (0, 200),
    "SR": (0, 1.25),
    "TOUT": (-5, 45),
    "WSR": (0, 35),
    "WDR": (0, 360)
}


# ============================
# 3. FUNCIÓN: ELIMINAR FUERA DE RANGO
# ============================

def clean_out_of_range(df):
    for col, (low, high) in ranges.items():
        if col in df.columns:
            df.loc[(df[col] < low) | (df[col] > high), col] = np.nan
    return df


# ============================
# 4. FUNCIÓN: IMPUTACIÓN COMPLETA
# ============================

def impute_df(df):

    # Asegurar índice de tiempo
    df.index = pd.to_datetime(df.index)

    # ----------------------------
    # Gases reactivos — interpolación temporal
    # ----------------------------
    gases_time = ["CO", "NO", "NO2", "NOX", "O3", "SO2"]
    for col in gases_time:
        if col in df.columns:
            df[col] = df[col].interpolate(method="time")

    # ----------------------------
    # Material particulado
    # ----------------------------
    if "PM10" in df.columns:
        df["PM10"] = df["PM10"].interpolate(method="linear")

    if "PM2.5" in df.columns:
        df["PM2.5"] = df["PM2.5"].interpolate(method="linear")

    # ----------------------------
    # Meteorológicas suaves
    # ----------------------------
    if "TOUT" in df.columns:
        df["TOUT"] = df["TOUT"].interpolate(method="spline", order=2)

    if "SR" in df.columns:
        df["SR"] = df["SR"].interpolate(method="time")

    # ----------------------------
    # Velocidad del viento — mediana móvil
    # ----------------------------
    if "WSR" in df.columns:
        df["WSR"] = df["WSR"].fillna(df["WSR"].rolling(5, min_periods=1).median())

    # ----------------------------
    # Dirección del viento — imputación circular
    # ----------------------------
    if "WDR" in df.columns:
        rad = np.deg2rad(df["WDR"])
        df["WDR_sin"] = np.sin(rad)
        df["WDR_cos"] = np.cos(rad)

        df["WDR_sin"] = df["WDR_sin"].interpolate(method="time")
        df["WDR_cos"] = df["WDR_cos"].interpolate(method="time")

        df["WDR"] = np.rad2deg(np.arctan2(df["WDR_sin"], df["WDR_cos"]))
        df["WDR"] = (df["WDR"] + 360) % 360

        df = df.drop(columns=["WDR_sin", "WDR_cos"])

    # ----------------------------
    # Limpieza final
    # ----------------------------
    df = df.ffill().bfill()
    df = df.round(2)

    return df


# ============================
# 5. PROCESAR CADA SHEET Y CREAR DF FINAL
# ============================

for name in sheets:
    temp_df = raw_data[name].copy()

    # Usar fecha como índice
    temp_df["date"] = pd.to_datetime(temp_df["date"])
    temp_df = temp_df.set_index("date")

    # Quitar valores fuera de rango
    temp_df = clean_out_of_range(temp_df)

    # Imputar con tu lógica avanzada
    temp_df = impute_df(temp_df)

    # Crear la variable global df_2022_NOMBRE
    globals()[f"df_2022_{name}"] = temp_df

    print(f"Listo: df_2022_{name} creado (shape={temp_df.shape})")


print("\n--- PROCESO COMPLETO ---")


Listo: df_2022_CENTRO creado (shape=(8760, 12))
Listo: df_2022_SUROESTE creado (shape=(8760, 12))
Listo: df_2022_NORTE creado (shape=(8760, 12))
Listo: df_2022_SUROESTE2 creado (shape=(8760, 12))
Listo: df_2022_SURESTE3 creado (shape=(8760, 12))
Listo: df_2022_NORESTE2 creado (shape=(8760, 12))

--- PROCESO COMPLETO ---


## 2023

In [3]:
# ============================
# 1. CARGAR DATOS
# ============================

raw_data = pd.read_excel(
    "DATOS_HISTORICOS_2023.xlsx",
    sheet_name=sheets
)

# Remover columnas que no usarás
for name in sheets:
    raw_data[name] = raw_data[name].drop(columns=["RAINF","RH","PRS"])


# ============================
# 2. RANGOS PERMITIDOS
# ============================

ranges = {
    "CO": (0, 14),
    "NO": (0, 500),
    "NO2": (0, 175),
    "NOX": (0, 500),
    "O3": (0, 175),
    "PM10": (0, 900),
    "PM2.5": (0, 800),
    "SO2": (0, 250),
    "SR": (0, 1),
    "TOUT": (0, 45),
    "WSR": (0, 40),
    "WDR": (0, 360)
}


# ============================
# 3. FUNCIÓN: ELIMINAR FUERA DE RANGO
# ============================

def clean_out_of_range(df):
    for col, (low, high) in ranges.items():
        if col in df.columns:
            df.loc[(df[col] < low) | (df[col] > high), col] = np.nan
    return df


# ============================
# 4. FUNCIÓN: IMPUTACIÓN COMPLETA
# ============================

def impute_df(df):

    # Asegurar índice de tiempo
    df.index = pd.to_datetime(df.index)

    # ----------------------------
    # Gases reactivos — interpolación temporal
    # ----------------------------
    gases_time = ["CO", "NO", "NO2", "NOX", "O3", "SO2"]
    for col in gases_time:
        if col in df.columns:
            df[col] = df[col].interpolate(method="time")

    # ----------------------------
    # Material particulado
    # ----------------------------
    if "PM10" in df.columns:
        df["PM10"] = df["PM10"].interpolate(method="linear")

    if "PM2.5" in df.columns:
        df["PM2.5"] = df["PM2.5"].interpolate(method="linear")

    # ----------------------------
    # Meteorológicas suaves
    # ----------------------------
    if "TOUT" in df.columns:
        df["TOUT"] = df["TOUT"].interpolate(method="spline", order=2)

    if "SR" in df.columns:
        df["SR"] = df["SR"].interpolate(method="time")

    # ----------------------------
    # Velocidad del viento — mediana móvil
    # ----------------------------
    if "WSR" in df.columns:
        df["WSR"] = df["WSR"].fillna(df["WSR"].rolling(5, min_periods=1).median())

    # ----------------------------
    # Dirección del viento — imputación circular
    # ----------------------------
    if "WDR" in df.columns:
        rad = np.deg2rad(df["WDR"])
        df["WDR_sin"] = np.sin(rad)
        df["WDR_cos"] = np.cos(rad)

        df["WDR_sin"] = df["WDR_sin"].interpolate(method="time")
        df["WDR_cos"] = df["WDR_cos"].interpolate(method="time")

        df["WDR"] = np.rad2deg(np.arctan2(df["WDR_sin"], df["WDR_cos"]))
        df["WDR"] = (df["WDR"] + 360) % 360

        df = df.drop(columns=["WDR_sin", "WDR_cos"])

    # ----------------------------
    # Limpieza final
    # ----------------------------
    df = df.ffill().bfill()
    df = df.round(2)

    return df


# ============================
# 5. PROCESAR CADA SHEET Y CREAR DF FINAL
# ============================

for name in sheets:
    temp_df = raw_data[name].copy()

    # Usar fecha como índice
    temp_df["date"] = pd.to_datetime(temp_df["date"])
    temp_df = temp_df.set_index("date")

    # Quitar valores fuera de rango
    temp_df = clean_out_of_range(temp_df)

    # Imputar con tu lógica avanzada
    temp_df = impute_df(temp_df)

    # Crear la variable global df_2022_NOMBRE
    globals()[f"df_2023_{name}"] = temp_df

    print(f"Listo: df_2023_{name} creado (shape={temp_df.shape})")


print("\n--- PROCESO COMPLETO ---")


Listo: df_2023_CENTRO creado (shape=(8758, 12))
Listo: df_2023_SUROESTE creado (shape=(8758, 12))
Listo: df_2023_NORTE creado (shape=(8758, 12))
Listo: df_2023_SUROESTE2 creado (shape=(8758, 12))
Listo: df_2023_SURESTE3 creado (shape=(8758, 12))
Listo: df_2023_NORESTE2 creado (shape=(8758, 12))

--- PROCESO COMPLETO ---


## 2024

In [4]:
# ============================
# 1. CARGAR DATOS
# ============================

raw_data = pd.read_excel(
    "DATOS_HISTORICOS_2024.xlsx",
    sheet_name=sheets
)

# Remover columnas que no usarás
for name in sheets:
    raw_data[name] = raw_data[name].drop(columns=["RAINF","RH","PRS"])


# ============================
# 2. RANGOS PERMITIDOS
# ============================

ranges = {
    "CO": (0, 18),
    "NO": (0, 400),
    "NO2": (0, 130),
    "NOX": (0, 500),
    "O3": (0, 180),
    "PM10": (0, 999),
    "PM2.5": (0, 999),
    "SO2": (0, 150),
    "SR": (0, 1.26),
    "TOUT": (-4, 45.5),
    "WSR": (0, 38),
    "WDR": (0, 360)
}


# ============================
# 3. FUNCIÓN: ELIMINAR FUERA DE RANGO
# ============================

def clean_out_of_range(df):
    for col, (low, high) in ranges.items():
        if col in df.columns:
            df.loc[(df[col] < low) | (df[col] > high), col] = np.nan
    return df


# ============================
# 4. FUNCIÓN: IMPUTACIÓN COMPLETA
# ============================

def impute_df(df):

    # Asegurar índice de tiempo
    df.index = pd.to_datetime(df.index)

    # ----------------------------
    # Gases reactivos — interpolación temporal
    # ----------------------------
    gases_time = ["CO", "NO", "NO2", "NOX", "O3", "SO2"]
    for col in gases_time:
        if col in df.columns:
            df[col] = df[col].interpolate(method="time")

    # ----------------------------
    # Material particulado
    # ----------------------------
    if "PM10" in df.columns:
        df["PM10"] = df["PM10"].interpolate(method="linear")

    if "PM2.5" in df.columns:
        df["PM2.5"] = df["PM2.5"].interpolate(method="linear")

    # ----------------------------
    # Meteorológicas suaves
    # ----------------------------
    if "TOUT" in df.columns:
        df["TOUT"] = df["TOUT"].interpolate(method="spline", order=2)

    if "SR" in df.columns:
        df["SR"] = df["SR"].interpolate(method="time")

    # ----------------------------
    # Velocidad del viento — mediana móvil
    # ----------------------------
    if "WSR" in df.columns:
        df["WSR"] = df["WSR"].fillna(df["WSR"].rolling(5, min_periods=1).median())

    # ----------------------------
    # Dirección del viento — imputación circular
    # ----------------------------
    if "WDR" in df.columns:
        rad = np.deg2rad(df["WDR"])
        df["WDR_sin"] = np.sin(rad)
        df["WDR_cos"] = np.cos(rad)

        df["WDR_sin"] = df["WDR_sin"].interpolate(method="time")
        df["WDR_cos"] = df["WDR_cos"].interpolate(method="time")

        df["WDR"] = np.rad2deg(np.arctan2(df["WDR_sin"], df["WDR_cos"]))
        df["WDR"] = (df["WDR"] + 360) % 360

        df = df.drop(columns=["WDR_sin", "WDR_cos"])

    # ----------------------------
    # Limpieza final
    # ----------------------------
    df = df.ffill().bfill()
    df = df.round(2)

    return df


# ============================
# 5. PROCESAR CADA SHEET Y CREAR DF FINAL
# ============================

for name in sheets:
    temp_df = raw_data[name].copy()

    # Usar fecha como índice
    temp_df["date"] = pd.to_datetime(temp_df["date"])
    temp_df = temp_df.set_index("date")

    # Quitar valores fuera de rango
    temp_df = clean_out_of_range(temp_df)

    # Imputar con tu lógica avanzada
    temp_df = impute_df(temp_df)

    # Crear la variable global df_2022_NOMBRE
    globals()[f"df_2024_{name}"] = temp_df

    print(f"Listo: df_2024_{name} creado (shape={temp_df.shape})")


print("\n--- PROCESO COMPLETO ---")


Listo: df_2024_CENTRO creado (shape=(8784, 12))
Listo: df_2024_SUROESTE creado (shape=(8784, 12))
Listo: df_2024_NORTE creado (shape=(8782, 12))
Listo: df_2024_SUROESTE2 creado (shape=(8782, 12))
Listo: df_2024_SURESTE3 creado (shape=(8782, 12))
Listo: df_2024_NORESTE2 creado (shape=(8782, 12))

--- PROCESO COMPLETO ---


## 2025

In [5]:
# ============================
# 1. CARGAR DATOS
# ============================

raw_data = pd.read_excel(
    "DATOS_HISTORICOS_2025.xlsx",
    sheet_name=sheets
)

# Remover columnas que no usarás
for name in sheets:
    raw_data[name] = raw_data[name].drop(columns=["RAINF","RH","PRS"])


# ============================
# 2. RANGOS PERMITIDOS
# ============================

ranges = {
    "CO": (0, 10),
    "NO": (0, 350),
    "NO2": (0, 175),
    "NOX": (0, 400),
    "O3": (0, 185),
    "PM10": (0, 820),
    "PM2.5": (0, 350),
    "SO2": (0, 405),
    "SR": (0, 1.2),
    "TOUT": (-4.5, 45),
    "WSR": (0, 40),
    "WDR": (0, 360)
}


# ============================
# 3. FUNCIÓN: ELIMINAR FUERA DE RANGO
# ============================

def clean_out_of_range(df):
    for col, (low, high) in ranges.items():
        if col in df.columns:
            df.loc[(df[col] < low) | (df[col] > high), col] = np.nan
    return df


# ============================
# 4. FUNCIÓN: IMPUTACIÓN COMPLETA
# ============================

def impute_df(df):

    # Asegurar índice de tiempo
    df.index = pd.to_datetime(df.index)

    # ----------------------------
    # Gases reactivos — interpolación temporal
    # ----------------------------
    gases_time = ["CO", "NO", "NO2", "NOX", "O3", "SO2"]
    for col in gases_time:
        if col in df.columns:
            df[col] = df[col].interpolate(method="time")

    # ----------------------------
    # Material particulado
    # ----------------------------
    if "PM10" in df.columns:
        df["PM10"] = df["PM10"].interpolate(method="linear")

    if "PM2.5" in df.columns:
        df["PM2.5"] = df["PM2.5"].interpolate(method="linear")

    # ----------------------------
    # Meteorológicas suaves
    # ----------------------------
    if "TOUT" in df.columns:
        df["TOUT"] = df["TOUT"].interpolate(method="spline", order=2)

    if "SR" in df.columns:
        df["SR"] = df["SR"].interpolate(method="time")

    # ----------------------------
    # Velocidad del viento — mediana móvil
    # ----------------------------
    if "WSR" in df.columns:
        df["WSR"] = df["WSR"].fillna(df["WSR"].rolling(5, min_periods=1).median())

    # ----------------------------
    # Dirección del viento — imputación circular
    # ----------------------------
    if "WDR" in df.columns:
        rad = np.deg2rad(df["WDR"])
        df["WDR_sin"] = np.sin(rad)
        df["WDR_cos"] = np.cos(rad)

        df["WDR_sin"] = df["WDR_sin"].interpolate(method="time")
        df["WDR_cos"] = df["WDR_cos"].interpolate(method="time")

        df["WDR"] = np.rad2deg(np.arctan2(df["WDR_sin"], df["WDR_cos"]))
        df["WDR"] = (df["WDR"] + 360) % 360

        df = df.drop(columns=["WDR_sin", "WDR_cos"])

    # ----------------------------
    # Limpieza final
    # ----------------------------
    df = df.ffill().bfill()
    df = df.round(2)

    return df


# ============================
# 5. PROCESAR CADA SHEET Y CREAR DF FINAL
# ============================

for name in sheets:
    temp_df = raw_data[name].copy()

    # Usar fecha como índice
    temp_df["date"] = pd.to_datetime(temp_df["date"])
    temp_df = temp_df.set_index("date")

    # Quitar valores fuera de rango
    temp_df = clean_out_of_range(temp_df)

    # Imputar con tu lógica avanzada
    temp_df = impute_df(temp_df)

    # Crear la variable global df_2022_NOMBRE
    globals()[f"df_2025_{name}"] = temp_df

    print(f"Listo: df_2025_{name} creado (shape={temp_df.shape})")


print("\n--- PROCESO COMPLETO ---")

Listo: df_2025_CENTRO creado (shape=(4344, 12))
Listo: df_2025_SUROESTE creado (shape=(4202, 12))
Listo: df_2025_NORTE creado (shape=(4344, 12))
Listo: df_2025_SUROESTE2 creado (shape=(4343, 12))
Listo: df_2025_SURESTE3 creado (shape=(4343, 12))
Listo: df_2025_NORESTE2 creado (shape=(4343, 12))

--- PROCESO COMPLETO ---


## Compilación

In [7]:
# Concatenar DataFrames por estación a través de 2022–2025

pattern = re.compile(r"^df_(2022|2023|2024|2025)_(.+)$")

frames_by_station: dict[str, list[tuple[int, pd.DataFrame]]] = {}
# Importante: iterar sobre una copia estática de globals() para evitar
# "RuntimeError: dictionary changed size during iteration"
for var_name, obj in list(globals().items()):
    if isinstance(obj, pd.DataFrame):
        m = pattern.match(var_name)
        if m:
            year = int(m.group(1))
            station = m.group(2)
            frames_by_station.setdefault(station, []).append((year, obj))

combined_by_station: dict[str, pd.DataFrame] = {}
for station, items in frames_by_station.items():
    # Ordenar por año
    items_sorted = sorted(items, key=lambda x: x[0])
    frames = [df for _, df in items_sorted]

    # Alinear columnas (unión en el primer orden encontrado)
    ordered_union_cols: list[str] = []
    seen_cols: set[str] = set()
    for df in frames:
        for c in df.columns:
            if c not in seen_cols:
                seen_cols.add(c)
                ordered_union_cols.append(c)

    aligned_frames = [df.reindex(columns=ordered_union_cols) for df in frames]
    combined_df = pd.concat(aligned_frames, ignore_index=True, sort=False)

    var_name = f"df_{station}_22_25"  # Nota: guiones no válidos en nombres de variables
    globals()[var_name] = combined_df
    combined_by_station[station] = combined_df

    print(f"Creado {var_name}: shape={combined_df.shape} | años={[y for y, _ in items_sorted]}")

# Acceso también vía diccionario
df_22_25_by_station = combined_by_station
print(f"\nEstaciones combinadas: {list(df_22_25_by_station.keys())}")


Creado df_CENTRO_22_25: shape=(30646, 12) | años=[2022, 2023, 2024, 2025]
Creado df_SUROESTE_22_25: shape=(30504, 12) | años=[2022, 2023, 2024, 2025]
Creado df_NORTE_22_25: shape=(30644, 12) | años=[2022, 2023, 2024, 2025]
Creado df_SUROESTE2_22_25: shape=(30643, 12) | años=[2022, 2023, 2024, 2025]
Creado df_SURESTE3_22_25: shape=(30643, 12) | años=[2022, 2023, 2024, 2025]
Creado df_NORESTE2_22_25: shape=(30643, 12) | años=[2022, 2023, 2024, 2025]

Estaciones combinadas: ['CENTRO', 'SUROESTE', 'NORTE', 'SUROESTE2', 'SURESTE3', 'NORESTE2']
